# Plots of null geodesics in Kerr spacetime
## Computation with `kerrgeodesic_gw`

This Jupyter/SageMath notebook is relative to the lectures
[Geometry and physics of black holes](https://luth.obspm.fr/~luthier/gourgoulhon/bh16/).

It requires [SageMath](http://www.sagemath.org/) (version $\geq$ 8.2), with the package [kerrgeodesic_gw](https://github.com/BlackHolePerturbationToolkit/kerrgeodesic_gw) (version $\geq$ 0.3.2). To install the latter, simply run 
```
sage -pip install kerrgeodesic_gw
```
in a terminal.

In [1]:
version()

'SageMath version 9.2, Release Date: 2020-10-24'

First, we set up the notebook to use LaTeX-formatted display:

In [2]:
%display latex

and we ask for CPU demanding computations to be performed in parallel on 8 processes:

In [3]:
Parallelism().set(nproc=8)

A Kerr black bole is entirely defined by two parameters $(m, a)$, where $m$ is the black hole mass and $a$ is the black hole angular momentum divided by $m$.
In this notebook, we shall set $m=1$ and we denote the angular momentum parameter $a$ by the symbolic variable `a`, using `a0` for a specific numerical value:

In [4]:
a = var('a')
a0 = 0.95

The spacetime object is created as an instance of the class `KerrBH`:

In [5]:
from kerrgeodesic_gw import KerrBH
M = KerrBH(a)
print(M)

Kerr spacetime M


The Boyer-Lindquist coordinate $r$ of the event horizon:

In [6]:
rH = M.event_horizon_radius()
rH

sqrt(-a^2 + 1) + 1

In [7]:
rH0 = rH.subs({a: a0})
show(LatexExpr(r'r_+ = '), rH0)

r_+ =  1.31224989991992

In [8]:
show(LatexExpr(r'r_- = '),
     M.inner_horizon_radius().subs({a: a0}))

r_- =  0.687750100080080

The method `boyer_lindquist_coordinates()` returns the chart of Boyer-Lindquist coordinates `BL` and allows the user to instanciate the Python variables `(t, r, th, ph)` to the coordinates $(t,r,\theta,\phi)$:

In [9]:
BL.<t, r, th, ph> = M.boyer_lindquist_coordinates()
BL

Chart (M, (t, r, th, ph))

The metric tensor is naturally returned by the method `metric()`:

In [10]:
g = M.metric()
g.display()

g = -(a^2*cos(th)^2 + r^2 - 2*r)/(a^2*cos(th)^2 + r^2) dt*dt - 2*a*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dt*dph + (a^2*cos(th)^2 + r^2)/(a^2 + r^2 - 2*r) dr*dr + (a^2*cos(th)^2 + r^2) dth*dth - 2*a*r*sin(th)^2/(a^2*cos(th)^2 + r^2) dph*dt + (2*a^2*r*sin(th)^4 + (a^2*r^2 + r^4 + (a^4 + a^2*r^2)*cos(th)^2)*sin(th)^2)/(a^2*cos(th)^2 + r^2) dph*dph

## Spherical photon orbits

Functions $\ell(r_0)$ and $q(r_0)$ for spherical photon orbits:

In [11]:
r = var('r')
lsph(a, r) = (r^2*(3 - r) - a^2*(r + 1))/(a*(r -1))
lsph

(a, r) |--> -(a^2*(r + 1) + (r - 3)*r^2)/(a*(r - 1))

In [12]:
qsph(a, r) = r^3 / (a^2*(r - 1)^2) * (4*a^2 - r*(r - 3)^2)
qsph

(a, r) |--> -((r - 3)^2*r - 4*a^2)*r^3/(a^2*(r - 1)^2)

### $\theta$-turning points:

In [13]:
theta0(a, l, q) = arccos(sqrt(1/2*(1 - (l^2+q)/a^2 + sqrt((1 - (l^2+q)/a^2)^2 + 4*q/a^2))))
theta0

(a, l, q) |--> arccos(sqrt(1/2*sqrt(((l^2 + q)/a^2 - 1)^2 + 4*q/a^2) - 1/2*(l^2 + q)/a^2 + 1/2))

In [14]:
theta1(a, l, q) = arccos(sqrt(1/2*(1 - (l^2+q)/a^2 - sqrt((1 - (l^2+q)/a^2)^2 + 4*q/a^2))))
theta1

(a, l, q) |--> arccos(sqrt(-1/2*sqrt(((l^2 + q)/a^2 - 1)^2 + 4*q/a^2) - 1/2*(l^2 + q)/a^2 + 1/2))

### Orbit close to a critical one

Parameters of a spherical photon orbit at $r_0 = 1.6m$:

In [15]:
r0 = 1.6
L0 = lsph(a0, r0)
Q0 = qsph(a0, r0)
L0, Q0

(2.17105263157895, 5.97569713758080)

Chosen parameters for the geodesic:

In [16]:
L = 1.0005*L0
Q = Q0
L, Q

(2.17213815789474, 5.97569713758080)

In [17]:
theta0(a0, L, Q)

0.705698214575078

In [18]:
r_init = 10
P = M.point((0, r_init, pi/2, 0), name='P')
print(P)

Point P on the Kerr spacetime M


A geodesic is constructed by providing the range $[\lambda_{\rm min},\lambda_{\rm max}]$ of the affine parameter $\lambda$, the initial point and either 
 - (i) the Boyer-Lindquist components $(p^t_0, p^r_0, p^\theta_0, p^\phi_0)$ of the initial 4-momentum vector
   $p_0 = \left. \frac{\mathrm{d}x}{\mathrm{d}\lambda}\right| _{\lambda_{\rm min}}$,
 - (ii) the four integral of motions $(\mu, E, L, Q)$
 - or (iii) some of the components of $p_0$ along with with some integrals of motion. 

In [19]:
lmax = 40 # lambda_max

In [20]:
Li = M.geodesic([0, lmax], P, mu=0, E=1, L=L, Q=Q, r_increase=False,
                a_num=a0, name='Li', latex_name=r'\mathcal{L}', verbose=True)

Initial tangent vector: 


p = 1.24434095052687 d/dt - 0.957332501180665 d/dr + 0.0244452390816306 d/dth + 0.0238275767289736 d/dph

The curve was correctly set.
Parameters appearing in the differential system defining the curve are [a].


The numerical integration of the geodesic equation is performed via `integrate()`, by providing the integration step $\delta\lambda$:

In [21]:
Li.integrate(step=0.001, method='dopri5')
#Li.integrate(step=0.001)
Li.check_integrals_of_motion(0.999*lmax)

quantity,value,initial value,diff.,relative diff.
,,,,
,,,,
,,,,
,,,,


In [22]:
print("Final point: ")
show(BL[:], "=", BL(Li(0.999*lmax)))

Final point: 


(t, r, th, ph) '=' (101.07107973443017, 20.161310098552885, 2.1976531053940387, 24.08010047543905)

In [23]:
lplot = 0.6*lmax
print("max lambda (plot): ", lplot)
graph1 = Li.plot(prange=(0, lplot), plot_points=2000, thickness=3, color='green',
                display_tangent=True, scale=0.1, width_tangent=10, color_tangent='green', 
                plot_points_tangent=12, horizon_color='lightgrey') \
         + line([(0,0,-3), (0,0,3)], color='black', thickness=2)
graph1

max lambda (plot):  24.0000000000000


Graphics3d Object

In [24]:
L = 0.9995*L0
Q = Q0
L, Q

(2.16996710526316, 5.97569713758080)

In [25]:
theta0(a0, L, Q)

0.705187305835540

In [26]:
r_init = 10
P = M.point((0, r_init, pi/2, 0), name='P')
lmax = 18.35
Li = M.geodesic([0, lmax], P, mu=0, E=1, L=L, Q=Q, r_increase=False,
                a_num=a0, name='Li', latex_name=r'\mathcal{L}', verbose=True)

Initial tangent vector: 


p = 1.24434604925682 d/dt - 0.957376197390187 d/dr + 0.0244452390816306 d/dth + 0.0238061083923306 d/dph

The curve was correctly set.
Parameters appearing in the differential system defining the curve are [a].


In [27]:
Li.integrate(step=0.0002, method='dopri5')
Li.check_integrals_of_motion(0.999*lmax)

quantity,value,initial value,diff.,relative diff.
,,,,
,,,,
,,,,
,,,,


In [28]:
print("Final point: ")
show(BL[:], "=", BL(Li(0.999*lmax)))

Final point: 


(t, r, th, ph) '=' (87.210806383301, 1.3183348234993377, 0.7219288807981207, 28.284837513555612)

In [29]:
lplot = lmax
print("max lambda (plot): ", lplot)
graph2 = Li.plot(prange=(0, lplot), plot_points=2000, thickness=2, color='orange',
                 plot_horizon=False) 
graph2

max lambda (plot):  18.3500000000000


Graphics3d Object

In [30]:
graph1 + graph2

Graphics3d Object

### Critical orbit

In [31]:
L = L0
Q = Q0
L, Q

(2.17105263157895, 5.97569713758080)

In [32]:
theta0(a0, L, Q)

0.705442812649839

In [33]:
r_init = 40
P = M.point((0, r_init, pi/2, 0), name='P')
lmax = 60
Li = M.geodesic([0, lmax], P, mu=0, E=1, L=L, Q=Q, r_increase=False,
                a_num=a0, name='Li', latex_name=r'\mathcal{L}', verbose=True)

Initial tangent vector: 


p = 1.05256221223911 d/dt - 0.997052937411049 d/dr + 0.00152782744260191 d/dth + 0.00138733416507633 d/dph

The curve was correctly set.
Parameters appearing in the differential system defining the curve are [a].


In [34]:
Li.integrate(step=0.001, method='dopri5')
Li.check_integrals_of_motion(0.999*lmax)

quantity,value,initial value,diff.,relative diff.
,,,,
,,,,
,,,,
,,,,


In [35]:
print("Final point: ")
show(BL[:], "=", BL(Li(0.999*lmax)))

Final point: 


(t, r, th, ph) '=' (182.09291319940124,
 1.5999998251692245,
 0.7711657882207236,
 51.496648220185136)

In [36]:
lplot = lmax
print("max lambda (plot): ", lplot)
graph3 = Li.plot(prange=(30, lplot), plot_points=2000, thickness=3, color='green',
                display_tangent=True, scale=0.1, width_tangent=10, color_tangent='green', 
                plot_points_tangent=12, horizon_color='lightgrey') \
         + line([(0,0,-3), (0,0,3)], color='black', thickness=2)
graph3

max lambda (plot):  60


Graphics3d Object